In [ ]:
%mkdir dataset
%cd dataset
!gdown 1N93rb_uFqKRZ9naX8CXShFt5RJHOmjZH
!unzip -q rwf-2000.zip

In [ ]:
%cd ..

# Import libraries

In [1]:
import os
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models import resnet18
from tqdm import tqdm

os.environ["TOKENIZERS_PARALLELISM"] = "false"

import warnings

warnings.filterwarnings("ignore")

In [2]:
import logging

logging.getLogger().setLevel(logging.INFO)
logging.basicConfig(format="%(message)s", level=logging.INFO)
LOGGER = logging.getLogger("Torch-Cls")
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Model

In [3]:
class Model(nn.Module):
    def __init__(self, num_classes=2):
        super(Model, self).__init__()
        self.resnet = resnet18(pretrained=True)
        self.resnet.fc = nn.Sequential(nn.Linear(self.resnet.fc.in_features, 512))
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x_3d):
        # (bs, C, T, H, W) -> (bs, T, C, H, W)
        x_3d = x_3d.permute(0, 2, 1, 3, 4)

        features = []
        for t in range(x_3d.size(1)):
            out = self.resnet(x_3d[:, t, :, :, :])
            features.append(out)

        # average pooling
        out = torch.mean(torch.stack(features), 0)

        x = self.fc1(out)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [4]:
# Example of how to use the model
model = Model(num_classes=2)

# Check param
param = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model has {param} parameters")

# Test the model with a random input (batch_size, channels, frames, height, width)
inputs = torch.rand(1, 3, 16, 224, 224)

output = model(inputs)

print(output.shape)

Model has 11505090 parameters
torch.Size([1, 2])


In [5]:
del model, inputs, output

# Load dataset

In [6]:
class VideoDataset(Dataset):
    def __init__(self, root_dir, phase="train", transform=None, n_frames=None):
        """
        Args:
            root_dir (string): Directory with all the videos (each video as a subdirectory of frames).
            transform (callable, optional): Optional transform to be applied on a sample.
            n_frames (int, optional): Number of frames to sample from each video, uniformly. If None, use all frames.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.n_frames = n_frames
        self.phase = phase
        self.videos, self.labels = self._load_videos()

    def _load_videos(self):
        videos, labels = [], []
        class_id = 0

        video_folders = os.listdir(os.path.join(self.root_dir, self.phase))

        for folder in video_folders:
            video_paths = os.listdir(os.path.join(self.root_dir, self.phase, folder))

            for video_path in video_paths:
                video_folder = os.path.join(
                    self.root_dir, self.phase, folder, video_path
                )
                frames = sorted(
                    (os.path.join(video_folder, f) for f in os.listdir(video_folder)),
                    key=lambda f: int(
                        "".join(filter(str.isdigit, os.path.basename(f)))
                    ),
                )

                if self.n_frames:
                    frames = self._uniform_sample(frames, self.n_frames)

                videos.append(frames)
                labels.append(class_id)

            class_id += 1

        return videos, labels

    def _uniform_sample(self, frames, n_frames):
        """
        Helper method to uniformly sample n_frames from the frames list.
        """
        stride = max(1, len(frames) // n_frames)
        sampled = [frames[i] for i in range(0, len(frames), stride)]
        return sampled[:n_frames]

    def __len__(self):
        return len(self.videos)

    def __getitem__(self, idx):
        video_frames = self.videos[idx]
        label = self.labels[idx]
        images = []
        for frame_path in video_frames:
            image = Image.open(frame_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            images.append(image)

        # Stack images along new dimension (sequence length)
        data = torch.stack(images, dim=0)

        # Rearrange to have the shape (C, T, H, W)
        data = data.permute(1, 0, 2, 3)
        return data, label

In [7]:
BATCH_SIZE = 16
MAX_LEN = 15
IMAGE_SIZE = 224


transform = transforms.Compose(
    [
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
    ]
)

# Load dataset
train_dataset = VideoDataset(
    root_dir="./dataset/rwf-2000", phase="train", transform=transform, n_frames=MAX_LEN
)

val_dataset = VideoDataset(
    root_dir="./dataset/rwf-2000", phase="val", transform=transform, n_frames=MAX_LEN
)

# Count number of cpus
cpus = os.cpu_count()
print(f"Number of cpus: {cpus}")

# Create data loaders
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, num_workers=cpus, shuffle=True
)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, num_workers=cpus, shuffle=False
)

# test
for data, label in train_loader:
    print(data.shape, label)
    break

Number of cpus: 80
torch.Size([16, 3, 15, 224, 224]) tensor([1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0])


# Train model

In [8]:
def colorstr(*input):
    *args, string = input if len(input) > 1 else ("blue", "bold", input[0])
    colors = {
        "black": "\033[30m",  # basic colors
        "red": "\033[31m",
        "green": "\033[32m",
        "yellow": "\033[33m",
        "blue": "\033[34m",
        "magenta": "\033[35m",
        "cyan": "\033[36m",
        "white": "\033[37m",
        "bright_black": "\033[90m",  # bright colors
        "bright_red": "\033[91m",
        "bright_green": "\033[92m",
        "bright_yellow": "\033[93m",
        "bright_blue": "\033[94m",
        "bright_magenta": "\033[95m",
        "bright_cyan": "\033[96m",
        "bright_white": "\033[97m",
        "end": "\033[0m",  # misc
        "bold": "\033[1m",
        "underline": "\033[4m",
    }
    return "".join(colors[x] for x in args) + f"{string}" + colors["end"]

In [9]:
def train_model(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=25, device="cuda"
):
    """
    Function to train the model.

    Parameters:
    - model: The neural network model to train.
    - train_loader: DataLoader for the training set.
    - val_loader: DataLoader for the validation set.
    - criterion: The loss function.
    - optimizer: The optimization algorithm.
    - num_epochs: Number of epochs to train for.
    - device: The device to run the training on, 'cuda' or 'cpu'.

    Returns:
    - model: The trained model.
    """
    since = time.time()

    history = {
        "train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": [],
        "lr": [],
    }
    best_val_acc = 0.0

    # Send the model to the specified device
    model.to(device)

    # Loop over the dataset multiple times
    for epoch in range(num_epochs):
        LOGGER.info(colorstr(f"Epoch {epoch}/{num_epochs-1}:"))

        # Each epoch has a training and validation phase
        for phase in ["train", "val"]:
            if phase == "train":
                LOGGER.info(
                    colorstr("bright_yellow", "bold", "\n%20s" + "%15s" * 3)
                    % ("Training:", "gpu_mem", "loss", "acc")
                )
                model.train()
            else:
                LOGGER.info(
                    colorstr("bright_green", "bold", "\n%20s" + "%15s" * 3)
                    % ("Validation:", "gpu_mem", "loss", "acc")
                )
                model.eval()

            running_items = 0
            running_loss = 0.0
            running_corrects = 0

            # Use the appropriate data loader
            data_loader = train_loader if phase == "train" else val_loader

            _phase = tqdm(
                data_loader,
                total=len(data_loader),
                bar_format="{desc} {percentage:>7.0f}%|{bar:10}{r_bar}{bar:-10b}",
                unit="batch",
            )

            # Iterate over data.
            for inputs, labels in _phase:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                # Track history only in train
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_items += outputs.size(0)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                epoch_loss = running_loss / running_items
                epoch_acc = running_corrects / running_items

                mem = f"{torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0:.3g}GB"
                desc = ("%35s" + "%15.6g" * 2) % (
                    mem,
                    epoch_loss,
                    epoch_acc,
                )
                _phase.set_description_str(desc)

            if phase == "train":
                history["train_loss"].append(epoch_loss)
                history["train_acc"].append(epoch_acc.item())
            else:
                history["val_loss"].append(epoch_loss)
                history["val_acc"].append(epoch_acc.item())
                if epoch_acc > best_val_acc:
                    best_val_acc = epoch_acc
                    history["best_epoch"] = epoch

                print(f"Best val Acc: {best_val_acc:4f}")

    time_elapsed = time.time() - since
    history["INFO"] = (
        "Training complete in {:.0f}h {:.0f}m {:.0f}s with {} epochs - Best val Acc: {:4f}".format(
            time_elapsed // 3600,
            time_elapsed % 3600 // 60,
            time_elapsed % 60,
            num_epochs,
            best_val_acc,
        )
    )

    return model

In [10]:
# Example usage (assuming you have defined your criterion and optimizer):
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Model(num_classes=2)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

trained_model = train_model(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=50, device=device
)

Epoch 0/49:

           Training:        gpu_mem           loss            acc
                             5.91GB       0.682812       0.581875     100%|██████████| 100/100 [00:48<00:00,  2.07batch/s]

         Validation:        gpu_mem           loss            acc
                             5.91GB       0.676673         0.5725     100%|██████████| 25/25 [00:11<00:00,  2.16batch/s]
Epoch 1/49:

           Training:        gpu_mem           loss            acc


Best val Acc: 0.572500


                             5.91GB       0.648954          0.625     100%|██████████| 100/100 [00:50<00:00,  1.98batch/s]

         Validation:        gpu_mem           loss            acc
                             5.91GB        0.56152          0.695     100%|██████████| 25/25 [00:07<00:00,  3.27batch/s]
Epoch 2/49:

           Training:        gpu_mem           loss            acc


Best val Acc: 0.695000


                             5.91GB       0.628437        0.65125     100%|██████████| 100/100 [00:51<00:00,  1.95batch/s]

         Validation:        gpu_mem           loss            acc
                             5.91GB       0.583614           0.66     100%|██████████| 25/25 [00:09<00:00,  2.72batch/s]
Epoch 3/49:

           Training:        gpu_mem           loss            acc


...

Best val Acc: 0.730000


                             5.91GB       0.117322       0.959375     100%|██████████| 100/100 [00:44<00:00,  2.22batch/s]

         Validation:        gpu_mem           loss            acc
                             5.91GB       0.786972         0.7025     100%|██████████| 25/25 [00:06<00:00,  3.84batch/s]
Epoch 49/49:

           Training:        gpu_mem           loss            acc


Best val Acc: 0.730000


                             5.91GB       0.108877          0.965     100%|██████████| 100/100 [00:40<00:00,  2.45batch/s]

         Validation:        gpu_mem           loss            acc
                             5.91GB        1.18655          0.685     100%|██████████| 25/25 [00:10<00:00,  2.39batch/s]


Best val Acc: 0.730000
